In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd


from typing import List, Optional, Tuple

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import (
    EarlyStopping,
    ReduceLROnPlateau,
    TensorBoard,
    History,
)
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import set_random_seed

import tensorflow as tf

<IPython.core.display.Javascript object>

# Get data

In [3]:
data = load_digits(as_frame=True)

X = data["data"] / 256
y = data["target"]

<IPython.core.display.Javascript object>

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

<IPython.core.display.Javascript object>

# Create model


In [6]:
X_train.iloc[0]

pixel_0_0    0.000000
pixel_0_1    0.000000
pixel_0_2    0.042969
pixel_0_3    0.039062
pixel_0_4    0.046875
               ...   
pixel_7_3    0.054688
pixel_7_4    0.035156
pixel_7_5    0.007812
pixel_7_6    0.000000
pixel_7_7    0.000000
Name: 421, Length: 64, dtype: float64

<IPython.core.display.Javascript object>

In [13]:
set_random_seed(42)

model = Sequential(
    [
        Dense(
            units=5,
            input_shape=(64,),
            kernel_initializer="he_normal",
            activation="relu",
            use_bias=True,
            name="dense_1",
        ),
        Dense(
            units=10,
            kernel_initializer="he_normal",
            activation="softmax",
            name="dense_2",
            use_bias=True,
        ),
    ]
)


OPTIMIZER = Adam(learning_rate=0.01)
LOSS = SparseCategoricalCrossentropy()
METRICS = ["accuracy"]


model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 5)                 325       
                                                                 
 dense_2 (Dense)             (None, 10)                60        
                                                                 
Total params: 385
Trainable params: 385
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

# Train model

In [ ]:
def train(X_train, y_train, **kwargs) -> History:
    history = model.fit(X_train, y_train, **kwargs)
    return history

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)


reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=5,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)

In [ ]:
TRAIN_KWARGS = {
    "batch_size": 128,
    "validation_data": (X_test, y_test),
    "epochs": 300,
    "callbacks": [early_stopping, reduce_lr_on_plateau],
}


history = train(X_train, y_train, **TRAIN_KWARGS)